# Deep Learning HW01
Gerald Baulig 0780827 2019/11/03

In this assignment we are asked to implement a neural network from scratch,
including Backpropagation and Storastic Gradient Decent algorithms.
The assigment includes two tasks:

1. Implement a regression network, and
2. implement a classification network.

## 1. Implement a Regression Network

Implement a regression network for the engery efficiency dataset.
I guess this dataset descripes the termal exchange of a building to its environment.
The dataset has 2 Values (t):

- Heating load, and
- Cooling load

And 8 features:

- Relative compactness,
- Surface area,
- Wall area,
- Roof area,
- Overall hight,
- Orientation,
- Glazing area, and
- Glazing area distribution.

We are required to define a feature-vector out of these features and predict the heating load of the buildings, by minimizing the "sum-of-squares" error function,

$$
\begin{equation}
E(w) = \sum_{n=1}^{N}{(t_n - y(X_n;w))^2}
\tag{1}
\end{equation}
$$

...while the evaluation should be processed by the "root-mean-square" error (basically an Euclidean distance :-),

$$
\begin{equation}
E_{RMS}(w) = \frac{\sqrt{\sum_{n=1}^{N}{(t_n - y(x_n;w))^2}}}{\sqrt{N}}
\tag{2}
\end{equation}
$$

## 2. Implement a Classifier

Implement a classification network for the Ionosphere dataset.



# A Fully-Connected Neural Network from Scratch

First, let's define what a Neural Network is.
A Neural Network is a class that can train and validate the weight of a stack of layers, w.r.t several hyper-parameters.
However, all what the Neural Network needs to know about his own network is, where to get its output from. Furthermore, the hyper-parameters are only required in the training process, but only few of them in the validation process,
such that the class defination could look like that:

In [315]:
class NeuralNetwork:
    def __init__(self, train_set, val_set, cost_func, loss_func):
        self.train_set = train_set
        self.val_set = val_set
        self.cost = cost_func
        self.loss = loss_func
        pass
    
    def train(self, output_layer, lr, lr_dec):
        self.training = True
        for Y in output_layer.poll_forward():
            Z = np.squeeze(self.cost.f(self.Y, Y))
            dZ = self.cost.d(self.Y, Y)
            output_layer.push_backward(dZ, lr)
            lr -= lr * lr_dec
            yield Z 
        pass
    
    def val(self, output_layer):
        self.training = False
        for Y in output_layer.pool_forward():
            Z = self.loss.f(self.Y, Y)
            yield Z
        pass
    
    def gen_input(self):
        if self.training:
            for self.X, self.Y, self.epoch, self.step in self.train_set:
                yield self.X
        else:
            for self.X, self.Y, self.epoch, self.step in self.val_set:
                yield self.X
        pass

## Activation Functions


### ReLU

In [316]:
import numpy as np
class ReLU:
    f = lambda x: np.maximum(x,0).astype(float)
    d = lambda x: (x > 0).astype(float)

#test
print(ReLU.f(np.array([1,-1,5,-5,0])))
print(ReLU.d(np.array([1,-1,5,-5,0])))

[1. 0. 5. 0. 0.]
[1. 0. 1. 0. 0.]


### Sigmoid

In [317]:
import numpy as np
class Sigmoid:
    f = lambda x: 1 / (1 + np.exp(-x))
    def d(x):
        S = Sigmoid.f(x)
        return S * (1 - S)
    pass

#test
print(Sigmoid.f(np.array([1,-1,5,-5,0])))
print(Sigmoid.d(np.array([1,-1,5,-5,0])))

[0.73105858 0.26894142 0.99330715 0.00669285 0.5       ]
[0.19661193 0.19661193 0.00664806 0.00664806 0.25      ]


## Let's Define a Fully-Connected Layer



In [318]:
class FcLayer:
    def __init__(self,
                 weights,
                 bias,
                 act_func,
                 input_generator,
                 backprop,
                ):
        self.w = weights
        self.b = bias
        self.act = act_func
        self.input = input_generator
        self.backprop = backprop
        pass
    
    def poll_forward(self):
        '''Polls the output from the underlying layer. (Feed Forward)'''
        for self.x in self.input:
            self.y = self.act.f(np.dot(self.x, self.w) + self.b)
            yield self.y
    
    def push_backward(self, dZ, lr):
        '''Pushes the loss to the underlying layer. (Back Propagation)'''
        dZ = dZ * self.act.d(self.y)
        dw = np.dot(self.x.T, dZ)
        if self.backprop:
            self.backprop(np.dot(dZ, self.w.T), lr)
        self.w += dw * lr

## Loss Functions


In [319]:
import numpy as np
class SOS:
    f = lambda t,y: np.sum(np.power(t - y, 2))
    d = lambda t,y: (t - y) * 2
    
class RMS:
    f = lambda t,y: np.power(SOS.f(t, y) / t.size, 0.5)
    d = lambda t,y: (SOS.d(t, y) / t.size) * 0.5

#test
print("SOS.f", SOS.f(np.array([1,-2,5,-5,0]), np.array([1,-1,5,-9,0])))
print("SOS.d", SOS.d(np.array([1,-2,5,-5,0]), np.array([1,-1,5,-9,0])))
print("RMS.f", RMS.f(np.array([1,-2,5,-5,0]), np.array([1,-1,5,-9,0])))
print("RMS.d", RMS.d(np.array([1,-2,5,-5,0]), np.array([1,-1,5,-9,0])))

SOS.f 17
SOS.d [ 0 -2  0  8  0]
RMS.f 1.8439088914585775
RMS.d [ 0.  -0.2  0.   0.8  0. ]


# Regression for Energy Efficiency

## Load the Energy Efficiency Data

In [31]:
import numpy as np

def unpack_data():
    raw = np.genfromtxt('EnergyEfficiency_data.csv', skip_header=1, delimiter=',')
    data = 
    N = raw.shape[0]
    pre = data[:,:5]
    ori = np.zeros((N,4))
    ori[:,0] = data[:,5] == 2
    ori[:,1] = data[:,5] == 3
    ori[:,2] = data[:,5] == 4
    ori[:,3] = data[:,5] == 5
    glaz = data[:,6]
    glaz_dist = np.zeros((N,5))
    glaz_dist[:,0] = data[:,7] == 1
    glaz_dist[:,1] = data[:,7] == 2
    glaz_dist[:,2] = data[:,7] == 3
    glaz_dist[:,3] = data[:,7] == 4
    glaz_dist[:,4] = data[:,7] == 5
    labels = data[:,8:]
    
    return np.array([pre, ori]), labels

unpack_data()

ValueError: could not broadcast input array from shape (768,5) into shape (768)

In [327]:
import numpy as np

data = np.array([
    [0,0,0],
    [0,1,1],
    [1,0,1],
    [1,1,0]
])

def gen_data(data, epochs, batch_size):
    step = 0
    X = np.zeros((batch_size, data.shape[1]-1))
    Y = np.zeros((batch_size, 1))
    for e in range(epochs):
        for sample in data:
            i = step % batch_size
            X[i] = sample[:-1]
            Y[i] = sample[-1:]
            step += 1
            if not step % batch_size:
                yield X, Y, e, step
    pass

nn = NeuralNetwork(
    gen_data(data, 10, 1),
    gen_data(data, 2, 1),
    SOS,
    SOS
)

#fix the random seed for the reproducibility.
#(another hyper-param)
np.random.seed(1)   

Fc1 = FcLayer(np.random.rand(2,4), #weights
              np.random.rand(4), #bias
              ReLU, #act func
              nn.gen_input(),
              None
             )

Fc2 = FcLayer(np.random.rand(4,2),
              np.random.rand(2),
              ReLU,
              Fc1.poll_forward(),
              Fc1.push_backward
             )

Out = FcLayer(np.random.rand(2,1),
              np.random.rand(1),
              ReLU,
              Fc2.poll_forward(),
              Fc2.push_backward
             )

for Z in nn.train(Out, 0.1, 1e-5):
    print(Out.y, nn.Y, Z)
    pass

[[2.66017798]] [[0.]] 7.076546890472223
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0
[[0.]] [[0.]] 0.0
[[0.]] [[1.]] 1.0
[[0.]] [[1.]] 1.0
[[0.]] [[0.]] 0.0


In [321]:
a = [0,1,2,3]
a[-2:]

[2, 3]